In [1]:
# Add dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import requests
from config import weather_api_key
from citipy import citipy
import gmaps
from config import g_key

In [2]:
# Starting url for weather api 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=150)
lngs = np.random.uniform(low=-180.000, high=180.000, size=150)

In [4]:
coordinates = zip(lats, lngs)
# Create list for holding cities
cities = []

# Identify nearest city for each lat and lng combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, add city to list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count
print(len(cities))
print(cities[0])

111
koran


In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all cities in list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

        
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | koran
Processing Record 2 of Set 1 | general roca
Processing Record 3 of Set 1 | roald
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | tasiilaq
Processing Record 6 of Set 1 | lodja
Processing Record 7 of Set 1 | rongcheng
Processing Record 8 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 9 of Set 1 | qaanaaq
Processing Record 10 of Set 1 | wichita falls
Processing Record 11 of Set 1 | narsaq
Processing Record 12 of Set 1 | doume
Processing Record 13 of Set 1 | port elizabeth
Processing Record 14 of Set 1 | san luis
Processing Record 15 of Set 1 | emba
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | kavieng
Processing Record 18 of Set 1 | port alfred
Processing Record 19 of Set 1 | nikolskoye
Processing Record 20 of Set 1 | busselton
Processing Record 21 of Set 1 | filingue
Processing Record 22 of Set 1 | matinhos
Processing Reco

In [6]:
len(city_data)

101

In [7]:
# Convert city_data to a data frame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Koran,43.8103,18.5689,65.71,39,40,6.91,BA,light rain
1,General Roca,-39.0333,-67.5833,58.62,43,100,9.93,AR,overcast clouds
2,Roald,62.5833,6.1333,52.16,81,20,19.57,NO,few clouds
3,Rikitea,-23.1203,-134.9692,76.37,72,29,6.06,PF,scattered clouds
4,Tasiilaq,65.6145,-37.6368,37.56,73,98,11.74,GL,overcast clouds


In [8]:
# Reorder the columns City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, and Wind Speed
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Koran,BA,43.8103,18.5689,65.71,39,40,6.91,light rain
1,General Roca,AR,-39.0333,-67.5833,58.62,43,100,9.93,overcast clouds
2,Roald,NO,62.5833,6.1333,52.16,81,20,19.57,few clouds
3,Rikitea,PF,-23.1203,-134.9692,76.37,72,29,6.06,scattered clouds
4,Tasiilaq,GL,65.6145,-37.6368,37.56,73,98,11.74,overcast clouds


In [9]:
# Create output file
output_data_file = "WeatherPy_Database.csv"

# Export data into CSV file
city_data_df.to_csv(output_data_file, index_label="City_ID")